# Algorithm Implement

In [1252]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import deque
time_period = 15
class Actor_Critic_continuous(nn.Module):

    def __init__(self, state_size, action_size, hidden=[128, 16]):
        super(Actor_Critic_continuous, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=state_size, out_channels=32, kernel_size=(2,10), stride=1, padding=2)  # 16, 20, 20
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2,9), stride=1)  # 32, 9, 9
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(2,8), stride=1)  # 32, 9, 9
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(2,7), stride=1)  # 32, 9, 9
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(2,6), stride=1)  # 32, 9, 9
        self.pool = nn.AvgPool2d(kernel_size=(2,1))
        self.sf = nn.BatchNorm1d(1)
        self.fc1 = nn.Linear(1600, 256)
        self.mu = nn.Linear(256, action_size)
        self.logstd = nn.Linear(256, action_size)
        self.output = nn.Linear(256, 1)

    def forward(self, state):
        # print(state.shape)
        x = F.relu(self.pool(self.conv1(state)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.pool(self.conv3(x)))
        x = x.view(-1, 1600)
        x = F.relu(self.fc1(x))
        mu = self.mu(x)
        # mu = mu/mu.abs().max()
        # mu = mu/mu.std()
        mu = torch.clamp(mu, -1, 1)
        logstd = self.logstd(x)
        # logstd = logstd/logstd.abs().max()
        # logstd = logstd/logstd.std()
        logstd = torch.clamp(logstd, -1, 1)
        # print(logstd.shape)
        values = self.output(x)
        return mu, logstd, values

# Data Loading

In [1180]:
import yfinance as yf

In [1181]:
# import pandas as pd
# import pickle
# file = open('../../FinBert/stock_data_full.bin', 'rb')
# data = pickle.load(file)
# file.close()

# codes = ['AAPL','AMZN','C','GOOG','JPM','NFLX','PLTR']
# for i in range(len(codes)):
#     data[i]['symbol'] = codes[i]
    
    
# df = pd.read_csv('../../FinRL/concat_data.csv')
# df=df[['date', 'open', 'high', 'low', 'close', 'volume',
#        'positive', 'neutral', 'negative', 'tic']]
# df['date'] = [x[:10] for x in df['date']]
# df = df[(df['date']>='2022-01-01') & (df['date']<'2023-09-30')]


# data = pd.read_csv('../../min_data_adjust.csv')

# stock_data = data[data['symbol']=='AAPL']

# stock_df = df

# stock_df['pctchange'] = (stock_df['close'] - stock_df['open'])/stock_df['open']

# stock_df

In [1195]:
data['symbol'] = data['symbol'].replace({'FB':'META'})
codes = ['AAPL','AMZN','C','GOOG','JPM','NFLX','PLTR']
start_date = '2015-01-01'
end_date = '2023-09-30'
res = []


for ticker_symbol in tqdm(codes):
    ticker = yf.Ticker(ticker_symbol)
    df = ticker.history(start=start_date, end=end_date)
    df['symbol'] = ticker_symbol
    res.append(df)


price = pd.concat(res).reset_index()

price['Time'] = price['Date'].dt.strftime('%H:%M:%S')
price['Date'] = price['Date'].dt.strftime('%Y-%m-%d')

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:03<00:00,  2.13it/s]


In [1196]:
price.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'dividends',
       'stock Splits', 'tic', 'time']

stock_df = price

stock_df['pctchange'] = (stock_df['close'] - stock_df['open'])/stock_df['open']

# Technical Indicators

In [1197]:
from finta import TA

In [1198]:
stock_df['SMA42'] = TA.SMA(stock_df, 42)
stock_df['SMA5'] = TA.SMA(stock_df, 5)
stock_df['SMA15'] = TA.SMA(stock_df, 15)
stock_df['AO'] = TA.AO(stock_df)
stock_df['OVB'] = TA.OBV(stock_df)
stock_df[['VW_MACD','MACD_SIGNAL']] = TA.VW_MACD(stock_df)
stock_df['RSI'] = TA.RSI(stock_df)
stock_df['CMO'] = TA.CMO(stock_df)

In [1199]:
stock_df = stock_df.dropna()

In [1200]:
stock_df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'dividends',
       'stock Splits', 'tic', 'time', 'pctchange', 'SMA42', 'SMA5', 'SMA15',
       'AO', 'OVB', 'VW_MACD', 'MACD_SIGNAL', 'RSI', 'CMO'],
      dtype='object')

In [1201]:
stock_df

,date,open,high,low,close,volume,dividends,stock Splits,tic,time,pctchange,SMA42,SMA5,SMA15,AO,OVB,VW_MACD,MACD_SIGNAL,RSI,CMO
41,2015-03-04,32.275002,32.389999,32.080002,32.134998,126665200,0.0,0.0,AAPL,00:00:00,-0.004338,29.660893,32.2935,32.212500,2.068302,3.629299e+09,0.840172,0.907569,61.674222,16.904053
42,2015-03-05,32.145000,32.187500,31.440001,31.602501,226068400,0.0,0.0,AAPL,00:00:00,-0.016877,29.762560,32.0930,32.238000,1.863464,3.403230e+09,0.737045,0.873462,55.129117,-0.357584
43,2015-03-06,32.099998,32.342499,31.565001,31.650000,291368400,0.0,0.0,AAPL,00:00:00,-0.014019,29.883690,32.0000,32.240333,1.640912,3.694599e+09,0.631641,0.825095,55.581933,1.089903
44,2015-03-09,31.990000,32.392502,31.264999,31.785000,354114000,0.0,0.0,AAPL,00:00:00,-0.006408,30.007976,31.9025,32.241333,1.388302,4.048713e+09,0.545374,0.769148,56.912823,5.398851
45,2015-03-10,31.602501,31.805000,30.950001,31.127501,275426400,0.0,0.0,AAPL,00:00:00,-0.015030,30.107738,31.6600,32.186000,1.092853,3.773286e+09,0.435050,0.702326,49.183405,-14.708492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13956,2023-09-25,14.000000,14.240000,13.820000,14.220000,39693900,0.0,0.0,PLTR,00:00:00,0.015714,15.767143,14.4480,15.112667,-0.617915,1.578379e+10,-0.463565,-0.351624,39.992687,-26.437952
13957,2023-09-26,13.950000,14.170000,13.880000,13.960000,37276200,0.0,0.0,PLTR,00:00:00,0.000717,15.715000,14.2100,15.029333,-0.778344,1.574651e+10,-0.497551,-0.380809,37.933747,-32.229420
13958,2023-09-27,14.200000,14.960000,14.160000,14.850000,62780600,0.0,0.0,PLTR,00:00:00,0.045775,15.644524,14.2320,14.999333,-0.839497,1.580929e+10,-0.434217,-0.391491,47.834102,-1.758829
13959,2023-09-28,15.140000,15.960000,14.825000,15.770000,66590800,0.0,0.0,PLTR,00:00:00,0.041612,15.547619,14.5860,15.036667,-0.583971,1.587588e+10,-0.280468,-0.369286,55.700508,19.537373


In [1202]:
stock_df_train = stock_df[stock_df['date']<='2023-03-31'].groupby(['date','tic']).agg('mean')
stock_df_test = stock_df[stock_df['date']>'2023-03-31'].groupby(['date','tic']).agg('mean')

In [1203]:
stock_df_train

open        high         low       close      volume  \
date       tic                                                                
2015-01-02 AMZN   15.629000   15.737500   15.348000   15.426000  55664000.0   
           C      43.159724   43.421730   42.707166   43.080326  11137800.0   
           GOOG   26.378078   26.490770   26.133251   26.168653  28951268.0   
           JPM    48.606926   49.216662   48.520937   48.849258  12600000.0   
           NFLX   49.151428   50.331429   48.731430   49.848572  13475000.0   
...                     ...         ...         ...         ...         ...   
2023-03-31 C      44.981947   45.339103   44.721326   45.261879  16009200.0   
           GOOG  101.709999  104.190002  101.440002  104.000000  28108000.0   
           JPM   126.836959  127.502147  126.044598  127.472801  13105200.0   
           NFLX  340.049988  345.839996  337.200012  345.480011   5610200.0   
           PLTR    8.150000    8.480000    8.080000    8.450000  26174800.0   

                 dividends  stock Splits  pctchange       SMA42        SMA5  \
date       tic                                                                
2015-01-02 AMZN        0.0           0.0  -0.012989  174.576572  139.943202   
           C           0.0           0.0  -0.001840  133.536437  109.628068   
           GOOG        0.0           0.0  -0.007939   41.355863   37.417859   
           JPM         0.4           0.0   0.004986  131.159031  114.947855   
           NFLX        0.0           0.0   0.014184  145.297554  125.779178   
...                    ...           ...        ...         ...         ...   
2023-03-31 C           0.0           0.0   0.006223   47.062827   44.192352   
           GOOG        0.0           0.0   0.022515   97.767143  102.328000   
           JPM         0.0           0.0   0.005013  133.472352  126.302837   
           NFLX        0.0           0.0   0.015968  329.112620  333.423999   
           PLTR        0.0           0.0   0.036810    8.240476    8.172000   

                      SMA15         AO           OVB    VW_MACD  MACD_SIGNAL  \
date       tic                                                                 
2015-01-02 AMZN  163.887068 -32.937297  4.269892e+09 -14.337630    -4.502248   
           C     128.130022 -22.610535  1.472999e+10  -3.968441    -1.576620   
           GOOG   40.190733  -3.267078  1.405542e+10  -2.263946    -0.862821   
           JPM   128.850620 -16.411174  1.630996e+10  -4.832951    -0.364026   
           NFLX  139.637401 -17.408020  1.617827e+10 -10.969341    -2.724948   
...                     ...        ...           ...        ...          ...   
2023-03-31 C      43.448444  -2.565935  1.409875e+10  -0.810921    -0.988223   
           GOOG  101.462667   5.691132  1.609833e+10   2.133172     1.892734   
           JPM   126.053057  -6.367679  1.605016e+10  -1.543318    -1.732596   
           NFLX  315.100002  11.975267  1.658732e+10   2.304919    -2.456776   
           PLTR    8.110000  -0.085429  1.520911e+10  -0.042054    -0.088929   

                       RSI        CMO  
date       tic                         
2015-01-02 AMZN   5.208108 -93.128553  
           C      8.599917 -88.473478  
           GOOG  11.334367 -82.259645  
           JPM    8.731430 -87.593940  
           NFLX   5.763957 -91.532139  
...                    ...        ...  
2023-03-31 C     50.212374   8.194892  
           GOOG  59.969684  22.260362  
           JPM   46.424919  -2.999573  
           NFLX  64.006162  39.218645  
           PLTR  56.347404  17.931957  

[13015 rows x 17 columns]

In [1204]:
train_date = sorted([x[0] for x in stock_df_train.index])
test_date = sorted([x[0] for x in stock_df_test.index])

In [1205]:
stock_df_train.columns

Index(['open', 'high', 'low', 'close', 'volume', 'dividends', 'stock Splits',
       'pctchange', 'SMA42', 'SMA5', 'SMA15', 'AO', 'OVB', 'VW_MACD',
       'MACD_SIGNAL', 'RSI', 'CMO'],
      dtype='object')

# RL

In [1206]:
# indicators = ['open', 'high', 'low', 'close', 'volume', 'positive', 'neutral', 'negative','SMA42', 'SMA5', 'SMA15', 'AO', 'OVB','VW_MACD',
#        'MACD_SIGNAL', 'RSI', 'CMO']

# indicators = ['pctchange', 'volume', 'positive', 'neutral', 'negative','SMA42', 'SMA5', 'SMA15', 'AO', 'OVB','VW_MACD',
#        'MACD_SIGNAL', 'RSI', 'CMO']

# indicators = ['pctchange', 'volume', 'positive', 'neutral', 'negative']
# indicators = ['positive', 'neutral', 'negative']
indicators = ['pctchange', 'SMA42', 'SMA5', 'SMA15', 'AO', 'OVB', 'VW_MACD',
       'MACD_SIGNAL', 'RSI', 'CMO']

In [1207]:
stock_df_train

open        high         low       close      volume  \
date       tic                                                                
2015-01-02 AMZN   15.629000   15.737500   15.348000   15.426000  55664000.0   
           C      43.159724   43.421730   42.707166   43.080326  11137800.0   
           GOOG   26.378078   26.490770   26.133251   26.168653  28951268.0   
           JPM    48.606926   49.216662   48.520937   48.849258  12600000.0   
           NFLX   49.151428   50.331429   48.731430   49.848572  13475000.0   
...                     ...         ...         ...         ...         ...   
2023-03-31 C      44.981947   45.339103   44.721326   45.261879  16009200.0   
           GOOG  101.709999  104.190002  101.440002  104.000000  28108000.0   
           JPM   126.836959  127.502147  126.044598  127.472801  13105200.0   
           NFLX  340.049988  345.839996  337.200012  345.480011   5610200.0   
           PLTR    8.150000    8.480000    8.080000    8.450000  26174800.0   

                 dividends  stock Splits  pctchange       SMA42        SMA5  \
date       tic                                                                
2015-01-02 AMZN        0.0           0.0  -0.012989  174.576572  139.943202   
           C           0.0           0.0  -0.001840  133.536437  109.628068   
           GOOG        0.0           0.0  -0.007939   41.355863   37.417859   
           JPM         0.4           0.0   0.004986  131.159031  114.947855   
           NFLX        0.0           0.0   0.014184  145.297554  125.779178   
...                    ...           ...        ...         ...         ...   
2023-03-31 C           0.0           0.0   0.006223   47.062827   44.192352   
           GOOG        0.0           0.0   0.022515   97.767143  102.328000   
           JPM         0.0           0.0   0.005013  133.472352  126.302837   
           NFLX        0.0           0.0   0.015968  329.112620  333.423999   
           PLTR        0.0           0.0   0.036810    8.240476    8.172000   

                      SMA15         AO           OVB    VW_MACD  MACD_SIGNAL  \
date       tic                                                                 
2015-01-02 AMZN  163.887068 -32.937297  4.269892e+09 -14.337630    -4.502248   
           C     128.130022 -22.610535  1.472999e+10  -3.968441    -1.576620   
           GOOG   40.190733  -3.267078  1.405542e+10  -2.263946    -0.862821   
           JPM   128.850620 -16.411174  1.630996e+10  -4.832951    -0.364026   
           NFLX  139.637401 -17.408020  1.617827e+10 -10.969341    -2.724948   
...                     ...        ...           ...        ...          ...   
2023-03-31 C      43.448444  -2.565935  1.409875e+10  -0.810921    -0.988223   
           GOOG  101.462667   5.691132  1.609833e+10   2.133172     1.892734   
           JPM   126.053057  -6.367679  1.605016e+10  -1.543318    -1.732596   
           NFLX  315.100002  11.975267  1.658732e+10   2.304919    -2.456776   
           PLTR    8.110000  -0.085429  1.520911e+10  -0.042054    -0.088929   

                       RSI        CMO  
date       tic                         
2015-01-02 AMZN   5.208108 -93.128553  
           C      8.599917 -88.473478  
           GOOG  11.334367 -82.259645  
           JPM    8.731430 -87.593940  
           NFLX   5.763957 -91.532139  
...                    ...        ...  
2023-03-31 C     50.212374   8.194892  
           GOOG  59.969684  22.260362  
           JPM   46.424919  -2.999573  
           NFLX  64.006162  39.218645  
           PLTR  56.347404  17.931957  

[13015 rows x 17 columns]

In [1208]:
def get_full_data(x, date):
        full_df = pd.DataFrame(0, index = codes, columns = x.columns)
        full_df.loc[set(full_df.index).intersection(set(x.index))] = x.loc[set(full_df.index).intersection(set(x.index))]
        v = full_df.values.reshape(1,-1)
        # full_df['date']=date
        return [date]+list(v[0])
    
dates = np.unique([x[0] for x in stock_df_train.index])
res = []
for date in tqdm(dates):
    x = stock_df_train[indicators].loc[date]
    res.append(get_full_data(x, date))

# res = pd.concat(res).reset_index()
# res.columns = ['tic', 'open', 'high', 'low', 'close', 'volume', 'positive',
#        'neutral', 'negative', 'pctchange', 'date']
stock_df_train_ = pd.DataFrame(res).set_index(0)

100%|██████████████████████████████████████████████████████████████████████████████| 2076/2076 [00:22<00:00, 93.72it/s]


In [1209]:
dates = np.unique([x[0] for x in stock_df_test.index])
res = []
for date in tqdm(dates):
    x = stock_df_test[indicators].loc[date]
    res.append(get_full_data(x, date))
    
# res = pd.concat(res).reset_index()
# res.columns = ['tic', 'open', 'high', 'low', 'close', 'volume', 'positive',
#        'neutral', 'negative', 'pctchange', 'date']
stock_df_test_ = pd.DataFrame(res).set_index(0)

100%|███████████████████████████████████████████████████████████████████████████████| 125/125 [00:01<00:00, 104.36it/s]


In [1210]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

stock_df_train_1 = scaler.fit_transform(stock_df_train_)
stock_df_test_1 = scaler.transform(stock_df_test_)

In [1211]:
stock_df_train_ = pd.DataFrame(stock_df_train_1, index = stock_df_train_.index, columns = stock_df_train_.columns)
stock_df_test_ = pd.DataFrame(stock_df_test_1, index = stock_df_test_.index, columns = stock_df_test_.columns)

In [1213]:
stock_df_train_

,1,2,3,4,5,6,7,8,9,10,...,61,62,63,64,65,66,67,68,69,70
0,,,,,,,,,,,,,,,,,,,,,
2015-01-02,-0.054586,-1.466146,-1.470969,-1.470050,-0.179677,-0.766273,-0.257380,-0.274567,-3.545577,-0.336775,...,-0.007770,-0.263104,-0.420769,-0.314613,0.109232,-0.655662,0.132134,0.147173,-0.616680,0.160934
2015-01-05,-0.054586,-1.466146,-1.470969,-1.470050,-0.179677,-0.766273,-0.257380,-0.274567,-3.545577,-0.336775,...,-0.007770,-0.263104,-0.420769,-0.314613,0.109232,-0.655662,0.132134,0.147173,-0.616680,0.160934
2015-01-06,-0.054586,-1.466146,-1.470969,-1.470050,-0.179677,-0.766273,-0.257380,-0.274567,-3.545577,-0.336775,...,-0.007770,-0.263104,-0.420769,-0.314613,0.109232,-0.655662,0.132134,0.147173,-0.616680,0.160934
2015-01-07,-0.054586,-1.466146,-1.470969,-1.470050,-0.179677,-0.766273,-0.257380,-0.274567,-3.545577,-0.336775,...,-0.007770,-0.263104,-0.420769,-0.314613,0.109232,-0.655662,0.132134,0.147173,-0.616680,0.160934
2015-01-08,-0.054586,-1.466146,-1.470969,-1.470050,-0.179677,-0.766273,-0.257380,-0.274567,-3.545577,-0.336775,...,-0.007770,-0.263104,-0.420769,-0.314613,0.109232,-0.655662,0.132134,0.147173,-0.616680,0.160934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27,-0.783201,1.564306,1.670589,1.601326,1.235906,1.266727,1.313910,1.181866,0.457419,0.407637,...,-0.587073,-0.015254,0.239940,0.101268,0.111152,1.458012,0.084601,0.086554,1.666796,-0.015170
2023-03-28,-0.196797,1.570824,1.664145,1.609327,1.178852,1.244783,1.278904,1.220663,0.352866,0.251837,...,-0.327285,-0.014706,0.232729,0.100300,0.108949,1.455946,0.082398,0.088406,1.635729,-0.132230
2023-03-29,0.562108,1.577891,1.675769,1.619775,1.143293,1.269262,1.357176,1.268519,0.682144,0.728367,...,0.681941,-0.014222,0.232889,0.100611,0.107074,1.458079,0.087419,0.090924,1.811807,0.554987


In [1222]:
# class Stock_Env:
#     def __init__(self, initial_asset, data, cost, time, codes):
#         self.asset = initial_asset
#         self.cash = initial_asset
#         self.stock = np.zeros(len(codes))
#         self.stockvalue = np.zeros(len(codes))
#         self.data = data
#         self.time = time
#         self.cost = cost
#         self.history=[]
#         self.total_cost = 0
#         self.initial_asset = initial_asset
#         self.timeid = time_period
#         self.rowid = self.time[time_period]
#         self.codes = codes
#         self.action_space = len(codes)+1
    
#     def reset(self):
#         self.asset = self.initial_asset
#         self.cash = self.initial_asset
#         self.stock = np.zeros(len(codes))
#         self.stockvalue = np.zeros(len(codes))
#         self.history=[]
#         self.total_cost = 0    
#         temp_time = time_period
#         self.rowid = self.time[temp_time]
#         self.timeid = temp_time
#         temp = []
#         for i in range(time_period):
#             temp.append(list(self.get_full_data(self.data.loc[self.time[temp_time-time_period+i+1]]).values.reshape(-1)))       
#         return np.array(temp)
    
#     def get_full_data(self,x):
#         full_df = pd.DataFrame(0, index = self.codes, columns = x.columns)
#         full_df.loc[set(full_df.index).intersection(set(x.index))] = x.loc[set(full_df.index).intersection(set(x.index))]
#         return full_df
    
#     def step(self, action):
#         done = False
#         states = self.data.loc[self.rowid]        
#         self.timeid +=1
#         self.rowid = self.time[self.timeid]
#         if self.timeid == len(self.data)-1:
#             done = True
#         next_state = self.data.loc[self.rowid]
#         last_asset = self.asset
        
#         # Calculate the total assets at the beginning of the next day
#         for i, code in enumerate(self.codes):
#             if code in next_state.index:
#                 self.stockvalue[i] = self.stock[i]*next_state.loc[code]['open']
#         old_asset = self.cash + self.stockvalue.sum()
        
#         self.asset = old_asset
#         # Calculate the position for each stock and cash

#         action = np.exp(action)/np.exp(action).sum()
#         # Get the stock asset value, where the last value of action is the position of cash.
#         stockvalue_ = old_asset * (1-action[-1])
        
#         # Adjust the postion
#         for i, code in enumerate(codes):
#             if code in next_state.index:
#                 target_value = action[i]*old_asset
#                 distance = target_value - self.stockvalue[i]
#                 stock_distance = int(distance/(next_state.loc[code]['open']*(1+self.cost)))
#                 self.stock[i] += stock_distance
#                 self.cash = self.cash - distance - np.abs(stock_distance*self.cost*next_state.loc[code]['open'])
#                 self.stockvalue[i] = self.stock[i] * next_state.loc[code]['close']
        
#         # Calculate new asset
#         self.asset = self.stockvalue.sum() + self.cash
#         reward = (self.asset - last_asset)/last_asset
        
#         # Generate new states
#         temp = []
#         for i in range(time_period):
#             temp.append(list(self.get_full_data(self.data.loc[self.time[self.timeid-time_period+i+1]]).values.reshape(-1)))
#         return (np.array(temp), reward, done)
#         # return (self.data[self.rowid-time_period:self.rowid][indicators].values, reward, done)

In [1259]:
class Stock_Env:
    def __init__(self, initial_asset, data, cost, time, codes, record):
        self.asset = initial_asset
        self.cash = initial_asset
        self.stock = np.zeros(len(codes))
        self.stockvalue = np.zeros(len(codes))
        self.data = data
        self.time = np.unique(time)
        self.cost = cost
        self.history=[]
        self.total_cost = 0
        self.initial_asset = initial_asset
        self.timeid = time_period
        self.rowid = self.time[time_period]
        self.codes = codes
        self.action_space = len(codes)+1
        self.codeid = pd.DataFrame(range(len(codes)), index=codes)
        self.record = record
    
    def reset(self):
        self.asset = self.initial_asset
        self.cash = self.initial_asset
        self.stock = np.zeros(len(codes))
        self.stockvalue = np.zeros(len(codes))
        self.history=[]
        self.total_cost = 0    
        temp_time = time_period
        self.rowid = self.time[temp_time]
        self.timeid = temp_time
        temp = []
        return self.record.loc[self.time[temp_time+1-time_period:temp_time+1]].values
        # for i in range(time_period):
        #     temp.append(list(self.get_full_data(self.data.loc[self.time[temp_time-time_period+i+1]]).values.reshape(-1)))       
        # return np.array(temp)
    
    def get_full_data(self,x):
        full_df = pd.DataFrame(0, index = self.codes, columns = x.columns)
        full_df.loc[set(full_df.index).intersection(set(x.index))] = x.loc[set(full_df.index).intersection(set(x.index))]
        return full_df
    
    def step(self, action):
        done = False
        states = self.data.loc[self.rowid]        
        self.timeid +=1
        self.rowid = self.time[self.timeid]
        if self.timeid == len(self.time)-1:
            done = True
        next_state = self.data.loc[self.rowid]
        last_asset = self.asset
        idx = self.codeid.loc[next_state.index].values.reshape(1,-1)
        # Calculate the total assets at the beginning of the next day
        self.stockvalue[idx] = self.stock[idx].reshape(1,-1)*next_state['open'].values.reshape(1,-1)
        old_asset = self.cash + self.stockvalue.sum()
        
        self.asset = old_asset
        # Calculate the position for each stock and cash

        action = np.exp(action)/np.exp(action).sum()
        # Get the stock asset value, where the last value of action is the position of cash.
        stockvalue_ = old_asset * (1-action[-1])
        
        # Adjust the postion
        target_value = action*old_asset
        distance = target_value[:-1] - self.stockvalue
        stock_distance = (distance[idx].reshape(-1))/((next_state['open'].values*(1+self.cost)).astype(int).reshape(-1))
        self.stock[idx] += stock_distance
        self.cash = self.cash - distance[idx].sum() - np.abs(stock_distance*self.cost*next_state['open'].values).sum()
        self.stockvalue[idx] = self.stock[idx] * next_state['close'].values
            
        # Calculate new asset
        self.asset = self.stockvalue.sum() + self.cash
        reward = (self.asset - last_asset)/self.initial_asset
        
        # Generate new states
        temp = self.record.loc[self.time[self.timeid+1-time_period:self.timeid+1]].values
        # for i in range(time_period):
        #     temp.append(list(self.get_full_data(self.data.loc[self.time[self.timeid-time_period+i+1]]).values.reshape(-1)))
        return (temp, reward, done)
        # return (self.data[self.rowid-time_period:self.rowid][indicators].values, reward, done)

In [1260]:
# %%prun

# import warnings
# warnings.filterwarnings('ignore')
# from tqdm import tqdm
# #env = gym.make()
# env = Stock_Env(1000000, stock_df_train, 0.002, [x[0] for x in stock_df_train.index], codes,stock_df_train_)
# # env_test = Stock_Env(1000000, stock_df_test, 0.002, [x[0] for x in stock_df_test.index], codes)
# num_episode = 1
# max_t = 1000
# reward_log = []

# for _ in range(num_episode):
    
#     # initialize
#     env.reset()
#     t = 0
#     episodic_reward = 0
    
#     for t in tqdm(list(range(max_t))):
        
#         #env.render()
#         action = np.random.rand(len(codes)+1) # random action
#         _, reward, done = env.step(action)
#         episodic_reward += reward
#         if done:
#             break
    
#     reward_log.append(episodic_reward)

In [1261]:
import torch
import torch.optim as optim
import numpy as np

import math

            
class Agent_continuous:
    
    def __init__(self, state_size, action_size, lr, beta, eps, tau, gamma, device, hidden=[256, 256], share=False, mode='MC', use_critic=False, normalize=False):
        self.state_size = state_size
        self.action_size = action_size
        self.lr = lr
        self.beta = beta
        self.eps = eps
        self.tau = tau
        self.gamma = gamma
        self.device = device
        self.share = share
        self.mode = mode
        self.use_critic = use_critic
        self.normalize = normalize
        
        if self.share:
            self.Actor_Critic = Actor_Critic_continuous(self.state_size, self.action_size, hidden).to(self.device)
            self.optimizer = optim.Adam(self.Actor_Critic.parameters(), lr)
        else:
            self.Actor = Actor_continuous(state_size, action_size, hidden).to(self.device)
            self.Critic = Critic(state_size, hidden).to(self.device)
            self.actor_optimizer = optim.Adam(self.Actor.parameters(), lr)
            self.critic_optimizer = optim.Adam(self.Critic.parameters(), lr)
            
    def act(self, states):
        with torch.no_grad():
            states = torch.tensor(states, dtype=torch.float).to(self.device).reshape([1]+list(states.shape))
            if self.share:
                mu, logstd, _ = self.Actor_Critic(states)
            else:
                mu, logstd = self.Actor(states)
            # mu = mu/mu.abs().max()
            # logstd = logstd/logstd.abs().max()
            actions = torch.distributions.Normal(mu, logstd.exp()).sample()
            # actions = actions/actions.abs().max()
            actions = actions.cpu().numpy().reshape(-1)
        return actions
    
    def process_data(self, states, actions, rewards, dones, batch_size):
        states = torch.tensor(states, dtype=torch.float).to(self.device)
        states = states.reshape([-1, 1, 15, 70])
        actions.append(np.zeros(self.action_size))
        states = torch.tensor(states, dtype=torch.float).to(self.device)
        actions = torch.tensor(actions, dtype=torch.float).to(self.device).view(-1, self.action_size)
        dones = torch.tensor(dones, dtype=torch.float).to(self.device).view(-1, 1)
        
        #calculate log probabilities and state values
        N = states.size(0) # N-1 is the length of actions, rewards and dones
        log_probs = torch.zeros((N, self.action_size)).to(self.device)
        old_mu = torch.zeros((N, self.action_size)).to(self.device)
        old_logstd = torch.zeros((N, self.action_size)).to(self.device)
        step = math.ceil(N/batch_size)
        
        for ind in range(step):
            if self.share:
                mu, logstd, _ = self.Actor_Critic(states[ind*batch_size:(ind+1)*batch_size, :])
            else:
                mu, logstd = self.Actor(states[ind*batch_size:(ind+1)*batch_size, :])
            # mu = mu/mu.abs().max()
            # logstd = logstd/logstd.abs().max()
            distribution = torch.distributions.normal.Normal(mu, logstd.exp())
            log_probs[ind*batch_size:(ind+1)*batch_size, :] = distribution.log_prob(actions[ind*batch_size:(ind+1)*batch_size, :])
            # log_probs = log_probs/log_probs.abs().max()
            old_mu[ind*batch_size:(ind+1)*batch_size, :] = mu
            old_logstd[ind*batch_size:(ind+1)*batch_size, :] = logstd 
            
        log_probs = log_probs[:-1, :]# remove the last one, which corresponds to no actions
        old_mu = old_mu[:-1, :]
        old_logstd = old_logstd[:-1, :]
        actions = actions[:-1, :]
        log_probs = log_probs.sum(dim=1, keepdim=True)

        rewards = np.array(rewards) #r_t
        
        return states, actions, old_mu.detach(), old_logstd.detach(), log_probs.detach(), rewards, dones
    
    def learn(self, states, actions, old_mu, old_logstd, log_probs, rewards, dones):
        # print(states, actions, old_mu, old_logstd, log_probs, rewards, dones)
        if self.share:
            new_mu, new_logstd, state_values = self.Actor_Critic(states)
            new_mu = new_mu[:-1, :]
            new_logstd = new_logstd[:-1, :]
        else:
            new_mu, new_logstd = self.Actor(states)
            new_mu = new_mu[:-1, :]
            new_logstd = new_logstd[:-1, :]
            state_values = self.Critic(states)
        
        # new_mu = new_mu/new_mu.abs().max()
        # new_logstd = new_logstd/new_logstd.abs().max()
        new_distribution = torch.distributions.normal.Normal(new_mu, new_logstd.exp())
        new_log_probs = new_distribution.log_prob(actions)
        # new_log_probs = new_log_probs/new_log_probs.abs().max()
        new_log_probs = new_log_probs.sum(dim=1, keepdim=True)
        
        KL = new_logstd - old_logstd + (old_logstd.exp().pow(2) + (new_mu - old_mu).pow(2))/(2*new_logstd.exp().pow(2) + 1e-6) - 0.5
        KL = KL.sum(dim=1, keepdim=True)
        
        L = rewards.shape[0]
        with torch.no_grad():
            G = []
            return_value = 0
            if self.mode == 'MC':
                for i in range(L-1, -1, -1):
                    return_value = rewards[i] + self.gamma * return_value * (1-dones[i])
                    G.append(return_value)
                G = G[::-1]
                G = torch.tensor(G, dtype=torch.float).view(-1, 1).to(self.device)
            else:
                rewards = torch.tensor(rewards, dtype=torch.float).view(-1, 1).to(self.device)
                G = rewards + (1 - dones) * self.gamma * state_values[1:, :]
            
        Critic_Loss = 0.5*(state_values[:-1, :] - G).pow(2).mean()
        with torch.no_grad():
            if self.use_critic:
                G = G - state_values[:-1, :] # advantage
            for i in range(L-2, -1, -1):
                G[i] += G[i+1]*self.gamma*(1-dones[i])*self.tau # cumulated advantage
            if self.normalize:
                G = (G - G.mean()) / (G.std() + 0.00001)
        
        ratio = (new_log_probs - log_probs).exp()
        Actor_Loss1 = ratio * G
        Actor_Loss2 = torch.clamp(ratio, 1-self.eps, 1+self.eps) * G
        Actor_Loss = -torch.min(Actor_Loss1, Actor_Loss2)
        Actor_Loss += self.beta * KL

        Actor_Loss = Actor_Loss.mean()
        
        if self.share:
            Loss = Actor_Loss + Critic_Loss
            self.optimizer.zero_grad()
            Loss.backward()
            torch.nn.utils.clip_grad_norm_(self.Actor_Critic.parameters(), 1)
            self.optimizer.step()
        else:
            self.critic_optimizer.zero_grad()
            Critic_Loss.backward()
            torch.nn.utils.clip_grad_norm_(self.Critic.parameters(), 1)
            self.critic_optimizer.step()
            self.actor_optimizer.zero_grad()
            Actor_Loss.backward()
            torch.nn.utils.clip_grad_norm_(self.Actor.parameters(), 1)
            self.actor_optimizer.step()

In [1262]:
# agent = Agent(1, len(env.action_space), 0.0005, 0.99, 'cuda', 'TD', True, False)
env = Stock_Env(1000000, stock_df_train, 0.002, [x[0] for x in stock_df_train.index], codes, stock_df_train_)
env_test = Stock_Env(1000000, stock_df_test, 0.002, [x[0] for x in stock_df_test.index], codes, stock_df_test_)
agent = Agent_continuous(state_size=1, 
                             action_size=env.action_space, 
                             lr=0.001, 
                             beta=1, 
                             eps=0.1, 
                             tau=0.99, 
                             gamma=0.99, 
                             device='cuda',
                             share=True, 
                             mode='TD', 
                             use_critic=True, 
                             normalize=False)

In [1263]:
import warnings
warnings.filterwarnings('ignore')
#env = gym.make()
num_episode = 20000
max_t = 1000
reward_log = []
average_log = [] # monitor training process
eps = 1
eps_decay = 0.997
eps_min = 0.01
num_frame = 1
C = 4 # update weights every C steps
state_deque = deque(maxlen=1)

def validation(env, agent):
    rewards_log = []
    average_log = []
    episodic_reward = 0
    done = False
    t = 0
    state = env.reset()
    while not done and t < max_t:
        t += 1
        action = agent.act(state.reshape([1]+list(state.shape)))
        frame, reward, done = env.step(action)
        next_state = frame
        state = next_state.copy()
        episodic_reward += reward
    return env.asset


rewards_log = []
average_log = []
state_history = []
action_history = []
done_history = []
reward_history = []
def train(agent, env, n_episode, n_update=4, update_frequency=1, max_t=1500, scale=1):
    global rewards_log, average_log, state_history, action_history, done_history, reward_history
    for i in range(1, 1 + num_episode):
        episodic_reward = 0
        done = False
        frame = env.reset()
        # state_deque = deque(maxlen=num_frame)
        for _ in range(num_frame):
            state_deque.append(frame)
        state = np.stack(state_deque, axis=0)
        # state = np.expand_dims(state, axis=0)
        
        done = False
        t = 0
        
        if len(state_history) == 0:
            state_history.append(state)
        else:
            state_history[-1] = state

        while not done and t < max_t:
            action = agent.act(state)
            frame, reward, done = env.step(action)
            state_deque.append(frame)
            next_state = np.stack(state_deque, axis=0)
            # next_state = np.expand_dims(next_state, axis=0)
           
            episodic_reward += reward
            action_history.append(action)
            done_history.append(done)
            reward_history.append(reward * scale)
            state = next_state.copy()
            state_history.append(state)
            t += 1
            
            
        if i % update_frequency == 0:
            states, actions, old_mu, old_logstd, log_probs, rewards, dones = agent.process_data(state_history, action_history, reward_history, done_history, 64)
            # print(states, actions, old_mu, old_logstd, log_probs, rewards, dones)
            for _ in range(n_update):
                agent.learn(states, actions, old_mu, old_logstd, log_probs, rewards, dones)
            state_history = []
            action_history = []
            done_history = []
            reward_history = []
        
        rewards_log.append(episodic_reward)
        average_log.append(np.mean(rewards_log[-100:]))
        
        val_asset = validation(env_test, agent)

        rewards_log.append(episodic_reward)
        average_log.append(np.mean(rewards_log[-100:]))
        print('\rEpisode {}, Reward {:.3f}, Average Reward {:.3f}, Asset {:.2f}, Validation Asset {:.2f}'.format(i, episodic_reward, average_log[-1], env.asset, val_asset), end='')
        if i % 100 == 0:
            print()
            
    return rewards, average_log

In [ ]:
# rewards_log, _ = train(agent, env, 2000, max_t, 10)
rewards_log, _ = train(agent=agent, 
                       env=env, 
                       n_episode=2000, 
                       n_update=10, 
                       update_frequency=4, 
                       max_t=2076, 
                       scale=1)

Episode 100, Reward -0.810, Average Reward -0.831, Asset 189944.28, Validation Asset 1017328.66
Episode 200, Reward -0.781, Average Reward -0.843, Asset 219271.83, Validation Asset 969042.802
Episode 300, Reward -0.782, Average Reward -0.837, Asset 217885.22, Validation Asset 1010571.58
Episode 400, Reward -0.805, Average Reward -0.825, Asset 195183.17, Validation Asset 1037668.80
Episode 500, Reward -0.809, Average Reward -0.828, Asset 191051.02, Validation Asset 1079238.06
Episode 600, Reward -0.715, Average Reward -0.802, Asset 285094.55, Validation Asset 1039536.78
Episode 700, Reward -0.827, Average Reward -0.755, Asset 173355.32, Validation Asset 1053929.88
Episode 800, Reward -0.783, Average Reward -0.730, Asset 216745.54, Validation Asset 1035829.04
Episode 900, Reward -0.696, Average Reward -0.655, Asset 304164.58, Validation Asset 955261.941
Episode 1000, Reward -0.505, Average Reward -0.553, Asset 494553.56, Validation Asset 1020123.37
Episode 1100, Reward -0.557, Average Re